In [1]:
from torchvision import models
fcn = models.segmentation.fcn_resnet101(weights="FCN_ResNet101_Weights.DEFAULT").eval()

In [2]:
pwd

'/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp'

In [3]:
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
import torchvision.transforms as T
from tqdm import tqdm
from scipy.ndimage.filters import gaussian_filter

/var/folders/vq/3kk0fzh5509g0jkz5rt3zzsh0000gn/T/ipykernel_4002/3007012044.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [4]:
# Define the helper function
def decode_segmap(image, nc=21):
  
  label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])

  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  
  for l in range(0, nc):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2]
    
  rgb = np.stack([r, g, b], axis=2)
  return rgb

In [5]:
def get_bg(net, img, show=True, dev='cpu'):
    
    if not type(img)==PIL.Image.Image:
        img = Image.fromarray(img)
    
    
    if show: plt.figure(figsize=(15,5))
    
    if show: plt.subplot(1,3,1);plt.imshow(img); plt.axis('off');
    
    
    # Comment the Resize and CenterCrop for better inference results
    trf = T.Compose([T.Resize(640), 
                   #T.CenterCrop(224), 
                   T.ToTensor(), 
                   T.Normalize(mean = [0.485, 0.456, 0.406], 
                               std = [0.229, 0.224, 0.225])])
    inp = trf(img).unsqueeze(0).to(dev)
    out = net.to(dev)(inp)['out']
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
    rgb = decode_segmap(om)
    #plt.imshow(rgb); plt.axis('off'); plt.show()
    
    
    seg_resized = Image.fromarray(rgb).resize((512,512),resample=PIL.Image.NEAREST)
    bg = (np.array(seg_resized)==0).sum(axis=-1)==3
    img_copy = np.array(img).copy()
    img_copy[:,:,0][bg] = 0
    img_copy[:,:,1][bg] = 0
    img_copy[:,:,2][bg] = 0
    #Image.fromarray(img_copy)
    
    if show: plt.subplot(1,3,2);plt.imshow(rgb); plt.axis('off');
    
    if show: plt.subplot(1,3,3);plt.imshow(img_copy); plt.axis('off');
    
    
    if show: plt.show()
    
    return bg

In [6]:
def load_video(fn):
    import skvideo
    import skvideo.io
    videodata = skvideo.io.vread(fn)
    return videodata

In [7]:
def save_video(videodata,ofn='outvideo.mp4'):
    import skvideo
    import skvideo.io
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-codec': 'h264', '-filter:v' : 'fps=29.97'})

In [8]:
import os
indir = '/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp/stimuli_cropped/'
indir2 = '/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp/stimuli_normed_cb_tint_1/'
files = [file for file in os.listdir(indir) if file.endswith('.mp4')]
files.sort()
files[0:5]

['id1d1.mp4', 'id1d2.mp4', 'id1f1.mp4', 'id1f2.mp4', 'id1h1.mp4']

In [34]:
mean_bg_colors = np.array([load_video(os.path.join(indir2,file))[:,0:10,:,:][:,:,0:10,:].mean(axis=0).mean(axis=0).mean(axis=0) for file in tqdm(files)])



100%|███████████████████████████████████████████| 70/70 [00:09<00:00,  7.04it/s]


In [39]:
mean_bg_colors.mean(axis=0).astype(np.uint8)

array([24, 48, 80], dtype=uint8)

In [9]:
file = files[0]
file = 'id7f2.mp4'
video_data = load_video(os.path.join(indir,file))
video_data.shape

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


(72, 512, 512, 3)

In [40]:
def remove_bg(video_data,bg_r=24,bg_b=48,bg_g=80):
    
    bgs = [get_bg(fcn,video_data[i,:,:,:],show=False) for i in range(video_data.shape[0])]
    bgs = np.array(bgs)
    bgs.shape
    
    use_bg = bgs.mean(axis=0)>.8

    video_data = load_video(os.path.join(indir2,file)) # load new video
    video_data_copy = video_data.copy()
    video_data_copy.shape

    for f in range(video_data_copy.shape[0]):
        video_data_copy[f,:,:,0][use_bg]=bg_r #20
        video_data_copy[f,:,:,1][use_bg]=bg_b #50
        video_data_copy[f,:,:,2][use_bg]=bg_g #80

    return video_data_copy

In [41]:
for file in tqdm(files):
    video_data = load_video(os.path.join(indir,file))
    video_data_copy = remove_bg(video_data,bg_r=20,bg_b=50,bg_g=80)
    filename = file.replace('.mp4','')
    save_video(video_data_copy,ofn=f'./stimuli_bg_removed/{filename}_bg_removed_fixed_max_outline.mp4')

100%|████████████████████████████████████████| 70/70 [2:54:07<00:00, 149.25s/it]
